# How do we add our data csv files to a MySQL database


## Option 1: Through MySQL workbench


MySQL Workbench provides a user-friendly interface to import CSV data and create corresponding tables.

Steps:

1.  Open MySQL Workbench: Launch the application and connect to your desired database.
2.  Right-click on the database: In the navigation pane, right-click on the database where you want to create the table.
3.  Select "Table Data Import Wizard": This will open the import wizard.
4.  Choose CSV file: Select the CSV file you want to import.
5.  Select destination: You can either choose an existing table or create a new table.
6.  Configure import settings:
    - Table name: Specify the name for the new table.
    - Column names: Map the CSV columns to the table columns.
    - Data types: Define the data types for the columns.
    - Character set and encoding: Ensure correct character encoding.
    - Field and line terminators: Specify the delimiters used in your CSV file.
7.  Start import: Click the "Start import" button to begin the process.

**Additional Tips:**

- Preview data: You can preview the imported data before creating the table.
- Handle errors: MySQL Workbench provides options to handle errors during the import process.
- Large CSV files: For large datasets, consider importing data in batches or using command-line tools for better performance.
- Data cleaning: Clean and validate your CSV data before importing to ensure data quality.


## Option 2: Programatically


In [ ]:
import os
from pathlib import Path
from typing import List, Tuple
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError, IntegrityError, DataError
from dotenv import load_dotenv

def find_root_dir():
    """Find the root directory of the project."""
    current_dir = Path.cwd()
    while current_dir.name != 'notebooks':
        if current_dir.parent == current_dir:
            raise FileNotFoundError("Could not find project root directory")
        current_dir = current_dir.parent
    return current_dir.parent

def log_dropped_rows(df, subset, table_name):
    """Log the dropped rows into a CSV file."""
    dropped_rows = df[df.duplicated(subset=subset, keep=False)]
    if not dropped_rows.empty:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"../../data/processed/duplicates_{table_name}_{timestamp}.csv"
        dropped_rows.to_csv(filename, index=False)
        print(f"Dropped rows logged to: {filename}")

def clean_data(df, table_name):
    """Clean and transform data based on the table name."""
    if table_name == 'contacts':
        df['contact_channel_first'] = df['contact_channel_first'].map({
            'contact_me': 'contact_me', 
            'book_it': 'book_it', 
            'instant_book': 'instant_book'
        })
        df['guest_user_stage_first'] = df['guest_user_stage_first'].map({
            'past_booker': 'past_booker', 
            'new': 'new_user'
        })
    elif table_name == 'listings':
        df['room_type'] = df['room_type'].map({
            'Private room': 'private_room',
            'Entire home/apt': 'entire_home',
            'Shared room': 'shared_room'
        })
    return df

def upload_csv_to_table(engine, csv_file_path: str, table_name: str):
    """
    Upload data from a CSV file to a MySQL table using pandas with batch processing.

    :param engine: SQLAlchemy engine for database connection
    :param csv_file_path: Path to the CSV file
    :param table_name: Name of the table to insert data into
    """
    chunk_size = 10000  # Adjust this value based on your system's memory capacity
    total_rows = 0
    
    try:
        for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size):
            chunk = clean_data(chunk, table_name)
            
            if table_name == 'contacts':
                # Log the rows being dropped
                log_dropped_rows(chunk, ['id_guest_anon', 'id_host_anon', 'id_listing_anon'], table_name)
                chunk.drop_duplicates(subset=['id_guest_anon', 'id_host_anon', 'id_listing_anon'], 
                                      keep='first', inplace=True)
            elif table_name == 'users':
                log_dropped_rows(chunk, 'id_user_anon', table_name)
                chunk.drop_duplicates(subset='id_user_anon', keep='first', inplace=True)
            elif table_name == 'listings':
                log_dropped_rows(chunk, 'id_listing_anon', table_name)
                chunk.drop_duplicates(subset='id_listing_anon', keep='first', inplace=True)
            
            try:
                chunk.to_sql(table_name, engine, if_exists='append', index=False, 
                             method='multi', chunksize=1000)
                total_rows += len(chunk)
                print(f"Uploaded {len(chunk)} rows to {table_name}. Total rows: {total_rows}")
            except IntegrityError as e:
                print(f"Integrity error uploading chunk to {table_name}: {e}")
            except DataError as e:
                print(f"Data error uploading chunk to {table_name}: {e}")
            except SQLAlchemyError as e:
                print(f"SQLAlchemy error uploading chunk to {table_name}: {e}")
            except Exception as e:
                print(f"Unexpected error uploading chunk to {table_name}: {e}")
        
        print(f"Completed uploading data from {csv_file_path} to {table_name}. Total rows uploaded: {total_rows}")
    except pd.errors.EmptyDataError:
        print(f"Error: The file {csv_file_path} is empty.")
    except pd.errors.ParserError as e:
        print(f"Error parsing CSV file {csv_file_path}: {e}")
    except Exception as e:
        print(f"Unexpected error processing {csv_file_path}: {e}")

def setup_and_populate_database(table_schemas: List[Tuple[str, str]], csv_files: List[Tuple[str, str]]):
    """
    Set up the database, create tables, and populate them with CSV data.

    :param table_schemas: List of tuples, each containing (table_name, table_schema)
    :param csv_files: List of tuples, each containing (table_name, csv_file_path)
    """
    project_root = find_root_dir()
    load_dotenv(project_root / '.env')

    db_host = os.getenv('DB_HOST')
    db_user = os.getenv('DB_USER')
    db_password = os.getenv('DB_PASSWORD')
    db_name = os.getenv('DB_NAME')

    connection = None
    try:
        # Create the initial connection
        connection = mysql.connector.connect(
            host=db_host,
            user=db_user,
            password=db_password,
            auth_plugin='mysql_native_password'
        )
        cursor = connection.cursor()

        # Create the database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        print(f"Database '{db_name}' created or already exists.")
        cursor.execute(f"USE {db_name}")

        # Create tables
        for table_name, table_schema in table_schemas:
            cursor.execute(table_schema)
            print(f"Table '{table_name}' created successfully.")

        # Close the initial connection
        cursor.close()
        connection.close()

        # Create SQLAlchemy engine for pandas operations
        engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}?connect_timeout=300")

        # Upload CSV data
        for table_name, csv_file_path in csv_files:
            upload_csv_to_table(engine, csv_file_path, table_name)

    except mysql.connector.Error as e:
        print(f"MySQL Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
        print("Database operations completed.")

# Usage example
contacts_schema = """
CREATE TABLE IF NOT EXISTS contacts (
    id_contact INT AUTO_INCREMENT,
    id_guest_anon VARCHAR(36) NOT NULL,
    id_host_anon VARCHAR(36) NOT NULL,
    id_listing_anon VARCHAR(36) NOT NULL,
    ts_interaction_first DATETIME,
    ts_reply_at_first DATETIME,
    ts_accepted_at_first DATETIME,
    ts_booking_at DATETIME,
    ds_checkin_first DATE,
    ds_checkout_first DATE,
    m_guests INT,
    m_interactions INT,
    m_first_message_length_in_characters INT,
    contact_channel_first ENUM('contact_me', 'book_it', 'instant_book'),
    guest_user_stage_first ENUM('past_booker', 'new_user'),
    PRIMARY KEY (id_contact),
     KEY idx_guest_host_listing (id_guest_anon, id_host_anon, id_listing_anon)
);
"""

listings_schema = """
CREATE TABLE IF NOT EXISTS listings (
    id_listing_anon VARCHAR(36) NOT NULL,
    room_type VARCHAR(50),
    listing_neighborhood VARCHAR(50),
    total_reviews INT,
    PRIMARY KEY (id_listing_anon)
);
"""

users_schema = """
CREATE TABLE IF NOT EXISTS users (
    id_user_anon VARCHAR(36) NOT NULL,
    country VARCHAR(50),
    words_in_user_profile INT,
    PRIMARY KEY (id_user_anon)
);
"""

table_schemas = [
    ("contacts", contacts_schema),
    ("listings", listings_schema),
    ("users", users_schema)
]

csv_files = [
    ("contacts", "../../data/raw/contacts.csv"),
    ("listings", "../../data/raw/listings.csv"),
    ("users", "../../data/raw/users.csv")
]

setup_and_populate_database(table_schemas, csv_files)


---


# Can SQL help us analyzing the KPIs we defined during week 1?


## Booking Conversion Rate

Definition: The percentage of inquiries that result in a confirmed booking


In [ ]:
SELECT
    (SUM(CASE WHEN ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS booking_conversion_rate
FROM
  contacts;

## Average Response Time

Definition: The average time taken by hosts to respond to guest inquiries.


In [ ]:
SELECT
    AVG(TIMESTAMPDIFF(MINUTE, ts_interaction_first, ts_reply_at_first)) AS avg_response_time_minutes
FROM
  contacts
WHERE
  ts_reply_at_first IS NOT NULL;

## Host Acceptance Rate

Definition: The percentage of inquiries that are accepted by hosts.


In [ ]:
SELECT
    (SUM(CASE WHEN ts_accepted_at_first IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS host_acceptance_rate
FROM
  contacts;

## Average Booking Lead Time

Definition: The average time between inquiry and check-in date for successful bookings.


In [ ]:
SELECT
    AVG(DATEDIFF(ds_checkin_first, ts_interaction_first)) AS avg_booking_lead_time_days
FROM
  contacts
WHERE
  ts_booking_at IS NOT NULL;

## Channel Effectiveness Index

Definition: A comparison of booking conversion rates across different contact channels.


In [ ]:
SELECT
    contact_channel_first,
    (SUM(CASE WHEN ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS booking_conversion_rate
FROM
  contacts
GROUP BY
  contact_channel_first
ORDER BY
  booking_conversion_rate DESC;

## New User Conversion Rate

Definition: The booking conversion rate for new users compared to repeat users.

First, assume that a "new user" is determined by whether this is their first interaction (or if they have fewer than a certain number of prior interactions):


In [ ]:
SELECT
    user_type,
    (SUM(CASE WHEN ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS booking_conversion_rate
FROM (
    SELECT
        c.id_guest_anon,
        COUNT(c.id_guest_anon) OVER (PARTITION BY c.id_guest_anon ORDER BY c.ts_interaction_first) AS interaction_count,
        c.ts_booking_at,
        CASE
            WHEN COUNT(c.id_guest_anon) OVER (PARTITION BY c.id_guest_anon ORDER BY c.ts_interaction_first) = 1 THEN 'New User'
            ELSE 'Repeat User'
        END AS user_type
    FROM
      contacts AS c
) AS subquery
GROUP BY user_type;

## Message Engagement Rate

Definition: The correlation between message length/count and booking success.


In [ ]:
SELECT
    CASE
        WHEN m_first_message_length_in_characters < 50 THEN '< 50 characters'
        WHEN m_first_message_length_in_characters BETWEEN 50 AND 100 THEN '50-100 characters'
        WHEN m_first_message_length_in_characters > 100 THEN '> 100 characters'
    END AS message_length_range,
    (SUM(CASE WHEN ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS booking_conversion_rate,
    AVG(m_interactions) AS avg_interactions
FROM
  contacts
GROUP BY
  message_length_range
ORDER BY
  booking_conversion_rate DESC;

## Room Type Preference Index

Definition: The booking conversion rate for different room types.


In [ ]:
SELECT
  l.room_type,
  (SUM(CASE WHEN c.ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS booking_conversion_rate
FROM
  contacts AS c
INNER JOIN
  listings AS l
  ON c.id_listing_anon = l.id_listing_anon
GROUP BY
  l.room_type
ORDER BY
  booking_conversion_rate DESC;

---


# What kind of business questions can SQL help us answer?


MySQL offers a robust platform for conducting exploratory data analysis (EDA). Its ability to handle large datasets efficiently, coupled with its powerful SQL capabilities, makes it a preferred choice for many analysts.

Typically if the business questions are properly formulated we can extract quite a bit of information from the data.

Let's have a look at some questions:

- How many hosts are also guests?
- Are hosts that are also guests booking more often / easily?
- Do hosts that are also guests allow more bookings?
- Which locations are more popular for enquiries and bookings?
- What are the bookings Success rate of those locations?
- Which guest profiles are most likely to book?
- What are the most common communication channels used?
- What is the average response time of hosts?
- Which listings have the highest booking rates?
- Do longer initial messages from guests lead to higher booking rates?
- What are the most common reasons for unsuccessful bookings?
- How does guest origin influence booking behavior?
- What factors contribute to a host’s success rate?
- etc


### How many hosts are also guests?


In [ ]:
SELECT
  COUNT(DISTINCT h.id_host_anon) AS hosts_that_are_guests
FROM
  contacts AS h
INNER JOIN
  contacts AS g
ON (h.id_host_anon = g.id_guest_anon);

### Are hosts that are also guests booking more often/easily?


**Bookings by hosts who are also guests**


In [ ]:
SELECT
  AVG(booking_count) AS avg_bookings_hosts_as_guests
FROM (
  SELECT
    g.id_guest_anon,
    COUNT(g.ts_booking_at) AS booking_count
  FROM
    contacts AS g
  INNER JOIN
    contacts AS h
    ON g.id_guest_anon = h.id_host_anon
  GROUP BY
    g.id_guest_anon
) AS host_guest_bookings;


**Bookings by guests who are not hosts**


In [ ]:
SELECT
  AVG(booking_count) AS avg_bookings_other_guests
FROM (
  SELECT
    id_guest_anon,
    COUNT(ts_booking_at) AS booking_count
  FROM
    contacts
  WHERE id_guest_anon NOT IN (
    SELECT
      DISTINCT g.id_guest_anon
    FROM
      contacts AS g
    INNER JOIN
      contacts AS h
      ON g.id_guest_anon = h.id_host_anon
  )
  GROUP BY
    id_guest_anon
) AS other_guest_bookings;

### Do hosts that are also guests allow more bookings?


**Bookings allowed by hosts who are also guests**


In [ ]:
SELECT
  AVG(accepted_booking_count) AS avg_accepted_bookings_hosts_as_guests
FROM (
  SELECT
    h.id_host_anon,
    COUNT(h.ts_accepted_at_first) AS accepted_booking_count
  FROM
    contacts AS h
  INNER JOIN
    contacts AS g
    ON h.id_host_anon = g.id_guest_anon
  GROUP BY
    h.id_host_anon
) AS host_guest_accepted_bookings;

**Bookings allowed by other hosts**


In [ ]:
SELECT
  AVG(accepted_booking_count) AS avg_accepted_bookings_other_hosts
FROM (
  SELECT
    id_host_anon,
    COUNT(ts_accepted_at_first) AS accepted_booking_count
  FROM
    contacts
  WHERE
    id_host_anon NOT IN (
      SELECT
          DISTINCT h.id_host_anon
      FROM
        contacts AS h
      INNER JOIN
        contacts AS g
        ON h.id_host_anon = g.id_guest_anon
)
GROUP BY id_host_anon
) AS other_host_accepted_bookings;


### Which locations are more popular for enquiries and bookings?


**Popular locations for enquiries**


In [ ]:
SELECT
  l.listing_neighborhood,
  COUNT(*) AS enquiry_count
FROM
  contacts c
INNER JOIN
  listings l ON c.id_listing_anon = l.id_listing_anon
GROUP BY
  l.listing_neighborhood
ORDER BY
  enquiry_count DESC;

**Popular locations for bookings**


In [ ]:
SELECT
  l.listing_neighborhood,
  COUNT(c.ts_booking_at) AS booking_count
FROM
  contacts c
INNER JOIN
  listings l ON c.id_listing_anon = l.id_listing_anon
WHERE
  c.ts_booking_at IS NOT NULL
GROUP BY
  l.listing_neighborhood
ORDER BY
  booking_count DESC;

How do we put both together?


In [ ]:
WITH
  location_by_enquiry AS (
    SELECT
      l.id_listing_anon,
      l.listing_neighborhood,
      COUNT(*) AS enquiry_count
    FROM
      contacts c
    INNER JOIN
      listings l
      ON c.id_listing_anon = l.id_listing_anon
    GROUP BY
      1, 2
  ),
  location_by_booking AS (
    SELECT
      l.id_listing_anon,
      l.listing_neighborhood,
      COUNT(c.ts_booking_at) AS booking_count
    FROM
      contacts c
    INNER JOIN
      listings l
      ON c.id_listing_anon = l.id_listing_anon
    WHERE
      c.ts_booking_at IS NOT NULL
    GROUP BY
      1, 2
  )
SELECT
  e.listing_neighborhood,
  SUM(e.enquiry_count) AS enquiry_count,
  SUM(b.booking_count) AS booking_count,
  SUM(b.booking_count) / SUM(e.enquiry_count) * 100 AS conversions
FROM
  location_by_enquiry AS e
INNER JOIN
  location_by_booking AS b
  ON (e.id_listing_anon = b.id_listing_anon)
GROUP BY
 1
ORDER BY
    enquiry_count DESC;

### Success rate of those locations?


In [ ]:
SELECT
  l.listing_neighborhood,
  COUNT(c.ts_accepted_at_first) AS accepted_bookings,
  COUNT(c.id_listing_anon) AS total_enquiries,
  (COUNT(c.ts_accepted_at_first) / COUNT(c.id_listing_anon)) * 100 AS success_rate
FROM
  contacts AS c
INNER JOIN
  listings AS l
  ON c.id_listing_anon = l.id_listing_anon
GROUP BY
  l.listing_neighborhood
ORDER BY
  total_enquiries DESC;

### What types of properties are most popular?


In [ ]:
SELECT
  l.room_type,
  COUNT(c.ts_booking_at) AS booking_count
FROM
  contacts AS c
INNER JOIN
  listings AS l
  ON c.id_listing_anon = l.id_listing_anon
WHERE
  c.ts_booking_at IS NOT NULL
GROUP BY
  l.room_type
ORDER BY
  booking_count DESC;

**What about by location? Any difference?**


In [ ]:
SELECT
  l.listing_neighborhood,
  l.room_type,
  COUNT(c.ts_booking_at) AS booking_count
FROM
  contacts AS c
INNER JOIN
  listings AS l
  ON c.id_listing_anon = l.id_listing_anon
WHERE
  c.ts_booking_at IS NOT NULL
GROUP BY
  l.listing_neighborhood, l.room_type
ORDER BY
  booking_count DESC;

**Pivoting it**


In [ ]:
SELECT
  l.listing_neighborhood,
  SUM(CASE WHEN LOWER(l.room_type) = 'entire_home' AND c.ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) AS entire_place_bookings,
  SUM(CASE WHEN LOWER(l.room_type) = 'private_room' AND c.ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) AS private_room_bookings,
  SUM(CASE WHEN LOWER(l.room_type) = 'shared_room' AND c.ts_booking_at IS NOT NULL THEN 1 ELSE 0 END) AS shared_room_bookings
FROM 
  contacts AS c
INNER JOIN 
  listings AS l 
  ON c.id_listing_anon = l.id_listing_anon
GROUP BY 
  l.listing_neighborhood
ORDER BY 
  entire_place_bookings DESC;

### Which guest profiles are most likely to book?


**Query: Are guests with more detailed profiles (more words in their profile) more likely to book a listing?**


In [ ]:
SELECT
  CASE
    WHEN u.words_in_user_profile < 50 THEN 'Low Profile Detail'
    WHEN u.words_in_user_profile BETWEEN 50 AND 150 THEN 'Medium Profile Detail'
    ELSE 'High Profile Detail'
  END AS profile_detail,
  COUNT(c.ts_booking_at) AS booking_count
FROM
  contacts AS c
INNER JOIN
  users AS u
  ON c.id_guest_anon = u.id_user_anon
WHERE
  c.ts_booking_at IS NOT NULL
GROUP BY
  profile_detail
ORDER BY
  booking_count DESC;

### What percentage of each profile manage to book a property?

Query: Analyze guest profiles by the number of words in their profile to see which profiles book the most and calculate the booking success rate.


In [ ]:
-- Percentage of bookings by profile detail
SELECT
  profile_detail,
  COUNT(ts_booking_at) AS booking_count,
  COUNT(ts_booking_at) / (SELECT COUNT(*) FROM contacts) * 100 AS booking_percentage
FROM (
  SELECT
    u.id_user_anon,
    CASE
      WHEN u.words_in_user_profile < 50 THEN 'Low Profile Detail'
      WHEN u.words_in_user_profile BETWEEN 50 AND 150 THEN 'Medium Profile Detail'
      ELSE 'High Profile Detail'
    END AS profile_detail,
    c.ts_booking_at
  FROM
    contacts AS c
  INNER JOIN
    users AS u
    ON c.id_guest_anon = u.id_user_anon
) AS profile_bookings
WHERE
  ts_booking_at IS NOT NULL
GROUP BY
  profile_detail
ORDER BY
  booking_percentage DESC;

### What are the most common communication channels used?

Query: Which contact channels are most commonly used by guests for initial interaction?


In [ ]:
SELECT
  contact_channel_first,
  COUNT(*) AS usage_count
FROM
  contacts
GROUP BY
  contact_channel_first
ORDER BY
  usage_count DESC;

### What are the most common communication channels used by the country of the guest?

Query: Determine which communication channels guests from different countries use most often for their initial contact.


In [ ]:
SELECT
  u.country,
  c.contact_channel_first,
  COUNT(*) AS usage_count
FROM 
  contacts AS c
INNER JOIN 
  users AS u 
  ON c.id_guest_anon = u.id_user_anon
GROUP BY 
  u.country, c.contact_channel_first
ORDER BY 
  usage_count DESC;

**Pivoting**


In [ ]:
SELECT
  u.country,
  SUM(CASE WHEN (c.contact_channel_first = "contact_me") THEN 1 ELSE 0 END) AS contact_me,
  SUM(CASE WHEN (c.contact_channel_first = "book_it") THEN 1 ELSE 0 END) AS book_it,
  SUM(CASE WHEN (c.contact_channel_first = "instant_book") THEN 1 ELSE 0 END) AS instant_book,
  COUNT(*) AS usage_count
FROM
  contacts AS c
INNER JOIN
  users AS u
  ON c.id_guest_anon = u.id_user_anon
GROUP BY
  u.country
ORDER BY
  usage_count DESC;

### What is the average response time of hosts?

Query: How quickly do hosts respond to guest inquiries, and does this affect booking success?


In [ ]:
SELECT
  AVG(TIMESTAMPDIFF(MINUTE, ts_interaction_first, ts_reply_at_first)) AS avg_response_time_minutes
FROM
  contacts
WHERE
  ts_reply_at_first IS NOT NULL;

### Which listings have the highest booking rates?

Query: Which listings have the highest ratio of bookings to inquiries?


In [ ]:
SELECT
  l.id_listing_anon,
  l.listing_neighborhood,
  COUNT(c.ts_booking_at) / COUNT(c.ts_interaction_first) AS booking_rate
FROM
  contacts AS c
INNER JOIN
  listings AS l
ON c.id_listing_anon = l.id_listing_anon
GROUP BY
  l.id_listing_anon, l.listing_neighborhood
ORDER BY
  booking_rate DESC
LIMIT 30;

### Do longer initial messages from guests lead to higher booking rates?

Query: Is there a correlation between the length of a guest's first message and the likelihood of a booking?


In [ ]:
SELECT
  CASE
    WHEN m_first_message_length_in_characters < 100 THEN 'Short Message'
    WHEN m_first_message_length_in_characters BETWEEN 100 AND 300 THEN 'Medium Message'
    ELSE 'Long Message'
  END AS message_length_category,
  COUNT(ts_booking_at) AS booking_count
FROM 
  contacts
GROUP BY 
  message_length_category
ORDER BY 
  booking_count DESC;

### What are the most common reasons for unsuccessful bookings?

Query: Are there patterns in guest or host behavior that lead to failed bookings (e.g., lack of response, late replies)?


In [ ]:
SELECT
  contact_channel_first,
  guest_user_stage_first,
  COUNT(*) AS failed_bookings_count
FROM
  contacts
WHERE
  ts_booking_at IS NULL
GROUP BY
  contact_channel_first, guest_user_stage_first
ORDER BY
  failed_bookings_count DESC;


In [ ]:
SELECT
  contact_channel_first,
  SUM(CASE WHEN guest_user_stage_first = 'new_user' THEN 1 ELSE 0 END) AS new_users,
  SUM(CASE WHEN guest_user_stage_first = 'past_booker' THEN 1 ELSE 0 END) AS past_booker,
  SUM(CASE WHEN guest_user_stage_first IS NULL THEN 1 ELSE 0 END) AS null_values,
  -- Add more stages as needed
  COUNT(*) AS total_failed_bookings
FROM
  contacts
WHERE
  ts_booking_at IS NULL
GROUP BY
  contact_channel_first
ORDER BY
  total_failed_bookings DESC;


In [ ]:
SELECT
  contact_channel_first,
  ROUND(SUM(CASE WHEN guest_user_stage_first = 'new_user' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS new_user_percentage,
  ROUND(SUM(CASE WHEN guest_user_stage_first = 'past_booker' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS past_booker_percentage,
  ROUND(SUM(CASE WHEN guest_user_stage_first IS NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS null_values_percentage,
  COUNT(*) AS total_failed_bookings
FROM
  contacts
WHERE
  ts_booking_at IS NULL
GROUP BY
  contact_channel_first
ORDER BY
  total_failed_bookings DESC;


### How does guest origin influence booking behavior?

Query: Do guests from certain countries tend to book more frequently or choose specific types of properties?


In [ ]:
SELECT
  u.country,
  COUNT(c.ts_booking_at) AS booking_count,
  l.room_type
FROM
  contacts AS c
INNER JOIN
  users AS u
  ON c.id_guest_anon = u.id_user_anon
INNER JOIN
  listings l
  ON c.id_listing_anon = l.id_listing_anon
WHERE
  c.ts_booking_at IS NOT NULL
GROUP BY
  u.country, l.room_type
ORDER BY
  booking_count DESC;

### What factors contribute to a host’s success rate?

Query: Are there specific characteristics (e.g., response time, listing type) that correlate with higher booking rates for hosts?


In [ ]:
SELECT
--   l.id_listing_anon,
  l.room_type,
  l.total_reviews,
  AVG(TIMESTAMPDIFF(MINUTE, c.ts_interaction_first, c.ts_reply_at_first)) AS avg_response_time_minutes,
  COUNT(c.ts_booking_at) AS booking_count
FROM
  contacts AS c
INNER JOIN
  listings AS l
  ON c.id_listing_anon = l.id_listing_anon
WHERE
  c.ts_booking_at IS NOT NULL
GROUP BY
  1, 2
ORDER BY
  booking_count DESC
LIMIT 30;